In [5]:
import torch

a= torch.dot(torch.tensor([2,3]),torch.tensor([2,1]))
torch.bmm(torch.FloatTensor(10,3,5),torch.FloatTensor(10,5,3)).size()

torch.Size([10, 3, 3])

In [5]:
import numpy as np
data=np.load("../data/cats/fid_files/cats_64.npz")

In [23]:
data["m"].shape

(2048,)

In [72]:
import torch
import math
import numpy as np
# num_step = 4, 배치 = 3, 크기는 = 8*8
def normalize_vecs(vectors: torch.Tensor) -> torch.Tensor:
    """
    Normalize vector lengths.
    """
    return vectors / (torch.norm(vectors, dim=-1, keepdim=True))

x, y = torch.meshgrid(torch.linspace(-1, 1, 8, ),
                          torch.linspace(-1, 1, 8))
x=x.T.flatten()
y=y.T.flatten()
z = -torch.ones_like(x) / np.tan((2 * math.pi * 30 / 360)/2)

ray_d_cam= normalize_vecs(torch.stack([x, y, z], -1))
z_vals=torch.linspace(2, 6, 4).reshape(1, 4, 1).repeat(64, 1, 1)
points = ray_d_cam.unsqueeze(1).repeat(1, 4, 1) * z_vals
points = torch.stack(3*[points]) 
z_vals = torch.stack(3*[z_vals]) 
ray_d_cam = torch.stack(3*[ray_d_cam]) 

In [ ]:
def transform_sampled_points(points, z_vals, ray_directions, h_stddev=1, v_stddev=1, h_mean=math.pi * 0.5, v_mean=math.pi * 0.5, mode='normal'):
    """Samples a camera position and maps points in camera space to world space."""

    n, num_rays, num_steps, channels = points.shape

    points, z_vals = perturb_points(points, z_vals, ray_directions)


    camera_origin, pitch, yaw = sample_camera_positions(n=points.shape[0], r=1, horizontal_stddev=h_stddev, vertical_stddev=v_stddev, horizontal_mean=h_mean, vertical_mean=v_mean, mode=mode)
    forward_vector = normalize_vecs(-camera_origin)

    cam2world_matrix = create_cam2world_matrix(forward_vector, camera_origin)

    points_homogeneous = torch.ones((points.shape[0], points.shape[1], points.shape[2], points.shape[3] + 1))
    points_homogeneous[:, :, :, :3] = points

    # should be n x 4 x 4 , n x r^2 x num_steps x 4
    transformed_points = torch.bmm(cam2world_matrix, points_homogeneous.reshape(n, -1, 4).permute(0,2,1)).permute(0, 2, 1).reshape(n, num_rays, num_steps, 4)


    transformed_ray_directions = torch.bmm(cam2world_matrix[..., :3, :3], ray_directions.reshape(n, -1, 3).permute(0,2,1)).permute(0, 2, 1).reshape(n, num_rays, 3)

    homogeneous_origins = torch.zeros((n, 4, num_rays))
    homogeneous_origins[:, 3, :] = 1
    transformed_ray_origins = torch.bmm(cam2world_matrix, homogeneous_origins).permute(0, 2, 1).reshape(n, num_rays, 4)[..., :3]

    return transformed_points[..., :3], z_vals, transformed_ray_directions, transformed_ray_origins, pitch, yaw


In [ ]:
def perturb_points(points, z_vals, ray_directions):
    distance_between_points = z_vals[:,:,1:2,:] - z_vals[:,:,0:1,:]
    offset = (torch.rand(z_vals.shape)-0.5) * distance_between_points
    z_vals = z_vals + offset

    points = points + offset * ray_directions.unsqueeze(2)
    return points, z_vals

perturb_points(points,z_vals,ray_d_cam)

In [80]:
z_vals.shape

torch.Size([3, 64, 4, 1])

In [26]:
import math
import numpy as np
z=-torch.ones_like(x)/np.tan((2 * math.pi * 30 / 360)/2)

def normalize_vecs(vectors: torch.Tensor) -> torch.Tensor:
    """
    Normalize vector lengths.
    """
    return vectors / (torch.norm(vectors, dim=-1, keepdim=True))

normalize_vecs(torch.stack([x, y, z], -1))

tensor([[[-0.2506, -0.2506, -0.9351],
         [-0.2545, -0.1818, -0.9498],
         [-0.2572, -0.1102, -0.9600],
         [-0.2586, -0.0369, -0.9653],
         [-0.2586,  0.0369, -0.9653],
         [-0.2572,  0.1102, -0.9600],
         [-0.2545,  0.1818, -0.9498],
         [-0.2506,  0.2506, -0.9351]],

        [[-0.1818, -0.2545, -0.9498],
         [-0.1847, -0.1847, -0.9653],
         [-0.1868, -0.1121, -0.9760],
         [-0.1878, -0.0376, -0.9815],
         [-0.1878,  0.0376, -0.9815],
         [-0.1868,  0.1121, -0.9760],
         [-0.1847,  0.1847, -0.9653],
         [-0.1818,  0.2545, -0.9498]],

        [[-0.1102, -0.2572, -0.9600],
         [-0.1121, -0.1868, -0.9760],
         [-0.1134, -0.1134, -0.9871],
         [-0.1140, -0.0380, -0.9928],
         [-0.1140,  0.0380, -0.9928],
         [-0.1134,  0.1134, -0.9871],
         [-0.1121,  0.1868, -0.9760],
         [-0.1102,  0.2572, -0.9600]],

        [[-0.0369, -0.2586, -0.9653],
         [-0.0376, -0.1878, -0.9815],
      

In [25]:
torch.stack([x,y,z],-1)

tensor([[[-1.0000, -1.0000, -3.7321],
         [-1.0000, -0.7143, -3.7321],
         [-1.0000, -0.4286, -3.7321],
         [-1.0000, -0.1429, -3.7321],
         [-1.0000,  0.1429, -3.7321],
         [-1.0000,  0.4286, -3.7321],
         [-1.0000,  0.7143, -3.7321],
         [-1.0000,  1.0000, -3.7321]],

        [[-0.7143, -1.0000, -3.7321],
         [-0.7143, -0.7143, -3.7321],
         [-0.7143, -0.4286, -3.7321],
         [-0.7143, -0.1429, -3.7321],
         [-0.7143,  0.1429, -3.7321],
         [-0.7143,  0.4286, -3.7321],
         [-0.7143,  0.7143, -3.7321],
         [-0.7143,  1.0000, -3.7321]],

        [[-0.4286, -1.0000, -3.7321],
         [-0.4286, -0.7143, -3.7321],
         [-0.4286, -0.4286, -3.7321],
         [-0.4286, -0.1429, -3.7321],
         [-0.4286,  0.1429, -3.7321],
         [-0.4286,  0.4286, -3.7321],
         [-0.4286,  0.7143, -3.7321],
         [-0.4286,  1.0000, -3.7321]],

        [[-0.1429, -1.0000, -3.7321],
         [-0.1429, -0.7143, -3.7321],
      

In [16]:
def get_initial_rays_trig(n, num_steps, fov, resolution, ray_start, ray_end):
    """Returns sample points, z_vals, and ray directions in camera space."""

    W, H = resolution
    # Create full screen NDC (-1 to +1) coords [x, y, 0, 1].
    # Y is flipped to follow image memory layouts.
    x, y = torch.meshgrid(torch.linspace(-1, 1, W),
                          torch.linspace(1, -1, H, )) # x,y grid , it descrete -1 to 1 with W resolution
    x = x.T.flatten() # transform하면 적절하게 x좌표에 맞는다.
    y = y.T.flatten() #  transform하면 잘 맞는다.
    z = -torch.ones_like(x) / np.tan((2 * math.pi * fov / 360)/2) # 

    rays_d_cam = normalize_vecs(torch.stack([x, y, z], -1))


    z_vals = torch.linspace(ray_start, ray_end, num_steps).reshape(1, num_steps, 1).repeat(W*H, 1, 1)
    points = rays_d_cam.unsqueeze(1).repeat(1, num_steps, 1) * z_vals

    points = torch.stack(n*[points])
    z_vals = torch.stack(n*[z_vals])
    rays_d_cam = torch.stack(n*[rays_d_cam])

    return points, z_vals, rays_d_cam

points,z_vals,rays_d_cam=get_initial_rays_trig(1,1,30,(8,8),2,6)


In [19]:
points.shape,z_vals.shape,rays_d_cam.shape

(torch.Size([1, 64, 1, 3]), torch.Size([1, 64, 1, 1]), torch.Size([1, 64, 3]))